# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-03-14","2019-03-15",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-03-14 to 2019-03-15
310


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Legendary\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Former Victoria's Secret executives reveal the...   
1  GameStop Investors Threaten Proxy Battle Over ...   
2     Is the retail industry enjoying a renaissance?   
3  A trade deal won't save the world's largest ec...   
4  US stocks move broadly higher, aiming for week...   

                                         description  
0  Victoria's Secret came under scrutiny in Novem...  
1  GameStop had tried to sell to a suitable buyer...  
2  Retailers are proving that, with innovation, i...  
3  There has been growing hope that the US and Ch...  
4  NEW YORK (AP) — U.S. stocks opened broadly hig...  


C:\Users\Legendary\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|██████████| 198/198 [00:35<00:00,  6.82it/s]


In [6]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|██████████| 198/198 [00:00<00:00, 330.86it/s]


tf Encoding


100%|██████████| 198/198 [00:00<00:00, 410.15it/s]


      ed     razek  relatively  unknown   outside  lingerie  industry  \
0    0.8  3.666667         0.1     0.25  0.058824  2.333333  0.045455   
1    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
2    0.0  0.000000         0.1     0.00  0.000000  0.000000  0.000000   
3    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
4    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
5    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
6    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
7    0.0  0.000000         0.0     0.00  0.117647  0.000000  0.022727   
8    0.0  0.000000         0.0     0.00  0.000000  0.000000  0.045455   
9    0.0  0.000000         0.1     0.00  0.000000  0.000000  0.068182   
10   0.0  0.000000         0.0     0.00  0.000000  0.000000  0.022727   
11   0.0  0.000000         0.0     0.00  0.000000  0.000000  0.000000   
12   0.0  0.000000         0.0     0.00  0.000000  

[198 rows x 13429 columns]


In [7]:
articleDB.head()

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,context,tags,tags_top_5,related_articles
0,0.372938,0.627062,https://www.businessinsider.com/victorias-secr...,1,Former Victoria's Secret executives reveal the...,Victoria's Secret came under scrutiny in Novem...,business-insider,2019-03-15T15:14:26Z,ed razek relatively unknown outside lingerie i...,Ed Razek was relatively unknown outside of the...,Ed Razek was relatively unknown outside of the...,"Razek, unknown outside, lingerie industry, las...","razek, unknown outside, lingerie industry, las...","transgender models,small store,dual role,unsha...","16, 22, 83"
1,0.384316,0.615684,http://fortune.com/2019/03/14/gamestop-proxy-b...,1,GameStop Investors Threaten Proxy Battle Over ...,GameStop had tried to sell to a suitable buyer...,fortune,2019-03-14T13:23:57Z,gamestop dealing with host problems business f...,GameStop is dealing with a host of problems in...,GameStop is dealing with a host of problems in...,"GameStop, major shareholders, GameStop board, ...","gamestop, major shareholders, gamestop board, ...","major shareholders,stock buybacks,potential su...","0, 38, 36"
2,0.469722,0.530278,https://www.ft.com/content/86acdcd8-20a7-11e9-...,1,Is the retail industry enjoying a renaissance?,"Retailers are proving that, with innovation, i...",financial-times,2019-03-14T23:05:18Z,see bad everywhere sears nine west shoes davi...,"We see the bad news everywhere: Sears, Nine We...","We see the bad news everywhere: Sears, Nine We...","Sears, West shoes, David, Bridal, House, Frase...","sears, west shoes, david, bridal, fraser, hmv,...","dotcom bust,other retailers,groceries sector,g...","19, 29, 180"
3,0.498526,0.501474,https://markets.businessinsider.com/news/stock...,1,A trade deal won't save the world's largest ec...,There has been growing hope that the US and Ch...,business-insider,2019-03-14T17:24:00Z,growing hope china could reach trade deal soo...,There has been growing hope that the US and Ch...,There has been growing hope that the US and Ch...,"China, trade deal, largest economies, trade wa...","china, trade deal, largest economies, trade wa...","agricultural energy,subject greater supply con...","12, 13, 99"
4,0.502574,0.497426,https://apnews.com/f8b7ff8a90fb4dab92fb9019695...,0,"US stocks move broadly higher, aiming for week...",NEW YORK (AP) — U.S. stocks opened broadly hig...,associated-press,2019-03-15T14:25:04Z,new york s stocks opened broadly higher wall s...,NEW YORK (AP) U.S. stocks opened broadly highe...,NEW YORK (AP) U.S. stocks opened broadly highe...,"YORK, U.S. stocks, higher Wall Street, retail ...","york, u.s. stocks, higher wall street, retail ...","largest economies,trade war,china,amazon,s&p","45, 47, 46"


In [8]:
articleDB.to_excel("March 14-15 data pull.xlsx")

In [9]:
import frontPage as fp
frontpage, articleDBB = fp.FrontPage(articleDB, trendingTermsDB)

                                               title               source  \
0  Former Victoria's Secret executives reveal the...     business-insider   
1  GameStop Investors Threaten Proxy Battle Over ...              fortune   
2     Is the retail industry enjoying a renaissance?      financial-times   
3  A trade deal won't save the world's largest ec...     business-insider   
4  US stocks move broadly higher, aiming for week...     associated-press   
5  MongoDB's cloud service is withstanding the th...                 cnbc   
6  Exclusive: Retailer Pier 1 taps debt restructu...              reuters   
7  The Daily 202: Trump will soon be forced to is...  the-washington-post   
8  EPA bans deadly chemical used in paint strippe...  the-washington-post   
9  The Health 202: Trump administration heads to ...  the-washington-post   

                   date                                        origContent  
0  2019-03-15T15:14:26Z  Ed Razek was relatively unknown outside of the... 